In [584]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import torch 

In [585]:
with open('../../sf_vs_nyc_house_data.csv', 'r') as f:
    labels = f.readline()

print(labels)

full_data = np.genfromtxt('../../sf_vs_nyc_house_data.csv', delimiter=',')[1:]
full_data.shape

in_sf,beds,bath,price,year_built,sqft,price_per_sqft,elevation



(492, 8)

In [586]:
X_train, X_test, y_train, y_test = train_test_split(full_data[:,1:], full_data[:,:1], test_size=.2)

print(f' Train shape X: {X_train.shape} y: {y_test.shape} \n  Test shape X: {X_test.shape}  y: {y_test.shape} ')

 Train shape X: (393, 7) y: (99, 1) 
  Test shape X: (99, 7)  y: (99, 1) 


---
NOW THAT WE HAVE THAT WE CAN START TO MAKE THE NN


In [587]:
# class NN_torch(torch.nn.Module):
    
#     def __init__(self):
#         super(NN_torch, self).__init__()

#         self.model = torch.nn.Sequential(
#             torch.nn.Linear(7,1),
#             torch.nn.ReLU(),
#             torch.nn.Linear(1,1)
#         )


In [588]:
class NN():

    def __init__(self, input_size, hidden_sizes, out_size) -> None:
        self.l1 = np.random.rand(input_size,hidden_sizes[0])
        self.b1 = np.zeros((1,hidden_sizes[0]))
        self.l2 = np.random.rand(hidden_sizes[0],out_size)
        self.b2 = np.zeros((1,out_size))

    def activation_func(self, o, type):
        if type.lower()== 'relu':
            return np.where(o >= 0, o, 0)
        if type.lower() == 'softmax':
            #make val stable
            o_stable = o - np.max(o, axis=1, keepdims=True)
            o_exp = np.exp(o_stable) #e^x for each x in 0_stable
            return o_exp / np.sum(o_exp, axis=1, keepdims=True)
        if type.lower() == 'sigmoid':
            return 1/(1+np.exp(-o))
        
    def forward(self, X):
        self.X = X
        self.o1 = X @ self.l1 + self.b1
        self.h1 = self.activation_func(self.o1, 'relu')
        self.o2 = self.h1 @ self.l2 + self.b2
        self.y_hat = self.activation_func(self.o2, 'sigmoid')
        return self.y_hat
    
    def calc_loss(self, y_true, y_hat, type):
        if type.lower() == 'mse':
            return np.sum((y_true - y_hat) ** 2) / len(y_true)

    def grad_relu(self, idx=None):
        if idx is None:
            return (self.h1 > 0).astype(float) #the as type flaot turn the bool mask in 0 or 1 which is what we wnat for the relu derivative
        else:
            return (self.h1[idx] > 0).astype(float)

    def grad_mse(self, y_true): #return vextor (training_samps x 1)
        return 2 * (self.y_hat - y_true) / len(y_true)

    def grad_sigmoid(self): #return vextor (training_samps x 1)
        return self.y_hat * (1 - self.y_hat)

    def dl_wrt_q0(self, y_true): #return vextor (training_samps x size of layer 2)
        #print(f'this is grad wrt_q0 doe the sconf layetr: {(self.grad_mse(y_true) * self.grad_sigmoid() * self.o1).shape}')
        return self.grad_mse(y_true) * self.grad_sigmoid() * self.o1

    def dl_wrt_b2(self, y_true):
        return self.grad_mse(y_true) * self.grad_sigmoid()

    def dl_wrt_b1(self, y_true):
        return self.grad_mse(y_true) * self.grad_sigmoid() * self.l2 * self.grad_relu()

    def grad_wrt_w(self, y_true):
        #(1 x training_samps) is the dl_wrt_b1(y_true).T then we multiply accordily by the X whis is (training_samps, 7) to ge full chainrule
        #if we did dl_wrt_b1(y_true).T * one specific row we would get the gradiet for each weight in layer one for one training exmple 
        #so the do for all and them summ across the colomns or jut the dpr product
        return np.dot(self.dl_wrt_b1(y_true).T, self.X)

    def backprop(self, alpha, y_true):
        # print(f'l1 before this : {self.l1},new gradit l1: {self.grad_wrt_w(y_true)}')
        # print(f'l2 before this : {self.l2},new gradit l2: {alpha * self.dl_wrt_q0(y_true)}')
        # print(f'b1 before this : {self.b1},new gradit l1: {alpha * self.dl_wrt_b1(y_true)}')
        # print(f'b2 before this : {self.b2},new gradit l1: {alpha * self.dl_wrt_b2(y_true)}\n\n')

        self.b2 -= alpha * np.sum(self.dl_wrt_b2(y_true), axis=0)
        self.l2 -= alpha * np.sum(self.dl_wrt_q0(y_true), axis=0)
        self.b1 -= alpha * np.sum(self.dl_wrt_b1(y_true), axis=0)
        self.l1 -= alpha * self.grad_wrt_w(y_true).reshape(self.l1.shape)




In [589]:
input_size = 7
hidden_sizes = [1]
out_size = 1
custom_nn = NN(7, [1], 1)

In [590]:
class NN_torch(torch.nn.Module):
    def __init__(self, input_size, hidden_sizes, out_size):
        super(NN_torch, self).__init__()
        self.l1 = torch.nn.Linear(input_size, hidden_sizes[0])
        self.l2 = torch.nn.Linear(hidden_sizes[0], out_size)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, X):
        o1 = self.l1(X)
        h1 = self.relu(o1)
        o2 = self.l2(h1)
        y_hat = self.sigmoid(o2)
        return y_hat

In [591]:
model = NN_torch(input_size, hidden_sizes, out_size)
with torch.no_grad():
    model.l1.weight = torch.nn.Parameter(torch.tensor(custom_nn.l1.T, dtype=torch.float32))
    model.l1.bias = torch.nn.Parameter(torch.tensor(custom_nn.b1.flatten(), dtype=torch.float32))
    model.l2.weight = torch.nn.Parameter(torch.tensor(custom_nn.l2.T, dtype=torch.float32))
    model.l2.bias = torch.nn.Parameter(torch.tensor(custom_nn.b2.flatten(), dtype=torch.float32))

In [592]:
# Verify the parameters are set correctly
print("l1 :\n", custom_nn.l1)
print("torch l1:\n", model.l1.weight.data.numpy().T)

print("b1 :\n", custom_nn.b1)
print("torch b1:\n", model.l1.bias.data.numpy())

print("l2 :\n", custom_nn.l2)
print("torch l2 :\n", model.l2.weight.data.numpy().T)

l1 :
 [[0.50778164]
 [0.4426363 ]
 [0.31200533]
 [0.07770886]
 [0.7090041 ]
 [0.51672924]
 [0.89778082]]
torch l1:
 [[0.5077816 ]
 [0.4426363 ]
 [0.31200534]
 [0.07770886]
 [0.7090041 ]
 [0.51672924]
 [0.89778084]]
b1 :
 [[0.]]
torch b1:
 [0.]
l2 :
 [[0.77104113]]
torch l2 :
 [[0.77104115]]


In [593]:
logits_torch = model.forward(torch.Tensor(X_train))

In [594]:

logits_custom = custom_nn.forward(X_train)
#logits_custom

In [595]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [596]:

alpha = 0.001
custom_nn.backprop(alpha, np.array(y_train))


loss = criterion(logits_torch, torch.Tensor([y_train]))

# Perform backpropagation and update weights
optimizer.zero_grad()
loss.backward()
print("\n\n Gradients for troucbh nn:")
for name, param in model.named_parameters():
    if param.grad is not None:
        print(f"{name} grad:\n{param.grad}")


optimizer.step()


# Verify the parameters are set correctly
print("\nl1 :",custom_nn.l1)
print("b1 :", custom_nn.b1)
print("l2 :", custom_nn.l2)
print("b2 :", custom_nn.b2)



print('\n\n',"Updated parameters:")
for name, param in model.named_parameters():
    print(name, param.data)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.